<a href="https://colab.research.google.com/github/sgc-aiml/DC-GAN-using-TPU/blob/main/Loading_TFDS_in_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import os

import tensorflow_datasets as tfds

##Define strategy

In [5]:
try:
  strategy == None
except:
  # strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  print("All devices: ", tf.config.list_logical_devices('TPU'))

  # optimizer = tf.tpu.CrossShardOptimizer(tf.train.GradientDescentOptimizer(0.01))

  strategy = tf.distribute.TPUStrategy(resolver)

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


##Load tfds

In [31]:

train_ds = tfds.load('mnist', split='train', as_supervised=True )
test_ds = tfds.load('mnist',split='test', as_supervised=True )


## Load tfds for TPU

In [ ]:

train_ds = tfds.load('mnist', split='train', as_supervised=True,try_gcs=True)
test_ds = tfds.load('mnist',split='test', as_supervised=True,try_gcs=True)


##Prepare the data

In [32]:
train_numpy = tfds.as_numpy(train_ds)
test_numpy = tfds.as_numpy(test_ds)

train_images = np.zeros((len(train_ds),28,28,1),dtype=np.uint8)
train_label = np.zeros((len(train_ds),))

test_images = np.zeros((len(test_ds),28,28,1),dtype=np.uint8)
test_label = np.zeros((len(test_ds),))


pos = 0
for image,label in train_numpy:
  train_images[pos,:,:,:] = image
  train_label[pos] = label
  pos = pos + 1

pos = 0
for image,label in test_numpy:
  test_images[pos,:,:,:] = image
  test_label[pos] = label
  pos = pos + 1
  

##Define the model

In [3]:
def make_classifier(fig_shape=(28,28,1)):
  cinputA = tf.keras.Input(shape=fig_shape)
  clayer = layers.Conv2D(32, kernel_size=4, strides=2, padding="same")(cinputA)
  clayer = layers.LeakyReLU(alpha=0.2)(clayer)

  clayer = layers.Conv2D(64, kernel_size=4, strides=2, padding="same")(clayer)
  clayer = layers.LeakyReLU(alpha=0.2)(clayer)

  # dlayer = layers.Conv2D(128, kernel_size=3, strides=2, padding="same")(dlayer)
  # dlayer = layers.LeakyReLU(alpha=0.2)(dlayer)
  clayer = layers.Flatten()(clayer)
  clayer = layers.Dropout(0.2)(clayer)

  clayer = layers.Dense(500)(clayer)
  clayer = layers.LeakyReLU(alpha=0.2)(clayer)
  clayer = layers.Dropout(0.2)(clayer)

  clayer2 = layers.Dense(10, activation="softmax")(clayer)

  classifier = tf.keras.models.Model(inputs=cinputA, outputs=clayer2)
  return classifier

##Train the model without TPU

In [ ]:
BUFFER_SIZE = len(train_ds)
BATCH_SIZE = 128
fig_shape = (28,28,1)
classifier_final = make_classifier(fig_shape)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False)
classifier_final.compile( optimizer="adam", loss=loss_object,metrics=['Accuracy'])
train_ds_batched = train_ds.batch(BATCH_SIZE)
classifier_final.fit(train_ds_batched,epochs=5)

##Train in TPU using tuple

In [47]:


with strategy.scope():
  # BUFFER_SIZE = len(train_ds)
  
  fig_shape = (28,28,1)
  classifier_final = make_classifier(fig_shape)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=False)
  classifier_final.compile( optimizer="adam", loss=loss_object,metrics=['Accuracy'])
  train_ds_batched = train_ds.batch(BATCH_SIZE)
  classifier_final.fit(train_ds_batched,epochs=5)
  


Epoch 1/5
469/469 [==============================] - 12s 17ms/step - loss: 0.5384 - Accuracy: 0.9259
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.1029 - Accuracy: 0.9700
Epoch 3/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0808 - Accuracy: 0.9759
Epoch 4/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0754 - Accuracy: 0.9780
Epoch 5/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0758 - Accuracy: 0.9786


##Train in TPU using numpy array

In [48]:
BUFFER_SIZE = len(train_ds)
BATCH_SIZE = 128

with strategy.scope():
  train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_label)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
  
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_label)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

with strategy.scope():
  
  BATCH_SIZE = 128
  fig_shape = (28,28,1)
  classifier_final = make_classifier(fig_shape)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=False,
      reduction=tf.keras.losses.Reduction.NONE)
  classifier_final.compile( optimizer="adam", loss=loss_object,metrics=['Accuracy'])

  classifier_final.fit(train_dataset,epochs=5)
  classifier_final.evaluate(test_dataset)
  


Epoch 1/5
469/469 [==============================] - 15s 22ms/step - loss: 0.6594 - Accuracy: 0.9176
Epoch 2/5
469/469 [==============================] - 8s 18ms/step - loss: 0.1080 - Accuracy: 0.9686
Epoch 3/5
469/469 [==============================] - 9s 18ms/step - loss: 0.0833 - Accuracy: 0.9758
Epoch 4/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0756 - Accuracy: 0.9787
Epoch 5/5
79/79 [==============================] - 5s 29ms/step - loss: 0.0986 - Accuracy: 0.9747
